# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f784413a280>
├── 'a' --> tensor([[ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410]])
└── 'x' --> <FastTreeValue 0x7f7844152ca0>
    └── 'c' --> tensor([[ 0.8413, -1.8009, -0.4463, -0.6287],
                        [ 1.3967,  0.7992, -1.1303, -0.5740],
                        [ 0.6748, -0.1954, -0.3331,  0.0111]])

In [4]:
t.a

tensor([[ 0.0617,  1.7817,  0.6672],
        [ 1.7408, -0.1656, -0.9410]])

In [5]:
%timeit t.a

68.6 ns ± 0.0366 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f784413a280>
├── 'a' --> tensor([[ 0.4685,  0.4251,  1.4190],
│                   [ 1.2686, -1.0916,  1.5418]])
└── 'x' --> <FastTreeValue 0x7f7844152ca0>
    └── 'c' --> tensor([[ 0.8413, -1.8009, -0.4463, -0.6287],
                        [ 1.3967,  0.7992, -1.1303, -0.5740],
                        [ 0.6748, -0.1954, -0.3331,  0.0111]])

In [7]:
%timeit t.a = new_value

65.1 ns ± 0.0252 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410]]),
    x: Batch(
           c: tensor([[ 0.8413, -1.8009, -0.4463, -0.6287],
                      [ 1.3967,  0.7992, -1.1303, -0.5740],
                      [ 0.6748, -0.1954, -0.3331,  0.0111]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0617,  1.7817,  0.6672],
        [ 1.7408, -0.1656, -0.9410]])

In [11]:
%timeit b.a

56.8 ns ± 0.0788 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1886,  0.5979,  1.0057],
               [ 0.9770, -0.2509, -0.1844]]),
    x: Batch(
           c: tensor([[ 0.8413, -1.8009, -0.4463, -0.6287],
                      [ 1.3967,  0.7992, -1.1303, -0.5740],
                      [ 0.6748, -0.1954, -0.3331,  0.0111]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.154 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 0.405 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 35.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 632 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f77743b41f0>
├── 'a' --> tensor([[[ 0.0617,  1.7817,  0.6672],
│                    [ 1.7408, -0.1656, -0.9410]],
│           
│                   [[ 0.0617,  1.7817,  0.6672],
│                    [ 1.7408, -0.1656, -0.9410]],
│           
│                   [[ 0.0617,  1.7817,  0.6672],
│                    [ 1.7408, -0.1656, -0.9410]],
│           
│                   [[ 0.0617,  1.7817,  0.6672],
│                    [ 1.7408, -0.1656, -0.9410]],
│           
│                   [[ 0.0617,  1.7817,  0.6672],
│                    [ 1.7408, -0.1656, -0.9410]],
│           
│                   [[ 0.0617,  1.7817,  0.6672],
│                    [ 1.7408, -0.1656, -0.9410]],
│           
│                   [[ 0.0617,  1.7817,  0.6672],
│                    [ 1.7408, -0.1656, -0.9410]],
│           
│                   [[ 0.0617,  1.7817,  0.6672],
│                    [ 1.7408, -0.1656, -0.9410]]])
└── 'x' --> <FastTreeValue 0x7f77746bd9d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 78.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f78400c35e0>
├── 'a' --> tensor([[ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410],
│                   [ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410],
│                   [ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410],
│                   [ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410],
│                   [ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410],
│                   [ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410],
│                   [ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410],
│                   [ 0.0617,  1.7817,  0.6672],
│                   [ 1.7408, -0.1656, -0.9410]])
└── 'x' --> <FastTreeValue 0x7f78400c3730>
    └── 'c' --> tensor([[ 0.8413, -1.8009, -0.4463, -0.6287],
                        [ 1.3967,  0.7992, -1.1303, -0.5740],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 78.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.9 µs ± 65.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0617,  1.7817,  0.6672],
                [ 1.7408, -0.1656, -0.9410]],
       
               [[ 0.0617,  1.7817,  0.6672],
                [ 1.7408, -0.1656, -0.9410]],
       
               [[ 0.0617,  1.7817,  0.6672],
                [ 1.7408, -0.1656, -0.9410]],
       
               [[ 0.0617,  1.7817,  0.6672],
                [ 1.7408, -0.1656, -0.9410]],
       
               [[ 0.0617,  1.7817,  0.6672],
                [ 1.7408, -0.1656, -0.9410]],
       
               [[ 0.0617,  1.7817,  0.6672],
                [ 1.7408, -0.1656, -0.9410]],
       
               [[ 0.0617,  1.7817,  0.6672],
                [ 1.7408, -0.1656, -0.9410]],
       
               [[ 0.0617,  1.7817,  0.6672],
                [ 1.7408, -0.1656, -0.9410]]]),
    x: Batch(
           c: tensor([[[ 0.8413, -1.8009, -0.4463, -0.6287],
                       [ 1.3967,  0.7992, -1.1303, -0.5740],
                       [ 0.6748, -0.1954, -0.3331,  0.0111]],
         

In [26]:
%timeit Batch.stack(batches)

80.2 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410],
               [ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410],
               [ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410],
               [ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410],
               [ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410],
               [ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410],
               [ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410],
               [ 0.0617,  1.7817,  0.6672],
               [ 1.7408, -0.1656, -0.9410]]),
    x: Batch(
           c: tensor([[ 0.8413, -1.8009, -0.4463, -0.6287],
                      [ 1.3967,  0.7992, -1.1303, -0.5740],
                      [ 0.6748, -0.1954, -0.3331,  0.0111],
                      [ 0.8413, -1.8009, -0.4463, -0.6287],
                      [ 1.3967,  

In [28]:
%timeit Batch.cat(batches)

150 µs ± 456 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

333 µs ± 4.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
